<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/Wikidata%E3%81%8B%E3%82%89%E4%BA%BA%E7%89%A9%E3%81%AE%E7%94%9F%E6%B2%A1%E5%B9%B4%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

準備

In [1]:
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 14.0 MB/s 
     |████████████████████████████████| 41 kB 502 kB/s 


設定（変更してください。）

In [2]:
# 対象
labels = [
    "足利尊氏",
    "足利義満",
    "足利義政",
    "源頼朝",
    "足利直冬"
]

# 出力先
final_output_path = "data/birth.json"

実行（変更しないでください。）

In [4]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import os

labels_path = "people.json"
tmp_dir = "tmp"

with open(labels_path, "w") as f:
    json.dump(labels, f, indent=4, ensure_ascii=False)

class WikidataTest:

    endpoint_url = "https://query.wikidata.org/sparql"

    def __init__(self, labels_path, tmp_dir, final_output_path):
        self.labels_path = labels_path
        self.tmp_dir = tmp_dir
        self.final_output_path = final_output_path

    def get_results(self, query):
        endpoint_url = self.endpoint_url
        user_agent = "WDQS-example Python/%s.%s" % (
            sys.version_info[0], sys.version_info[1])
        # TODO adjust user agent; see https://w.wiki/CX6
        sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()

    def sparql(self, label):

        query = f"""SELECT distinct *
        WHERE 
        {{
        ?item rdfs:label "{label}"@ja . 
        optional {{ ?item wdt:P569 ?year_of_birth }} 
        optional {{ ?item wdt:P570 ?year_of_death }} 
        }}"""

        results = self.get_results(query)

        return results

    def getPeople(self):

        with open(self.labels_path, "r") as f:
            people = json.load(f)

        self.people = people

    def load(self, label):
        tmp_dir = self.tmp_dir
        opath_wikidata = f"{tmp_dir}/wikidata/{label}.json"

        if not os.path.exists(opath_wikidata):

            df = self.sparql(label)

            os.makedirs(os.path.dirname(opath_wikidata), exist_ok=True)

            with open(opath_wikidata, "w") as f:
                json.dump(df, f, indent=4, ensure_ascii=False)

        with open(opath_wikidata, "r") as f:
            df = json.load(f)

        return df

    def getYearIntValue(self, result, key):
        if key not in result:
            return None
        value = result[key]["value"]
        year = value.split("-")[0]
        if "wikidata" not in year:
            return int(year)
        return None

    def main(self):
        self.getPeople()
        people = self.people

        results = {}

        keys = ["year_of_birth", "year_of_death"]

        for label in tqdm(people):

            if label is None:
                continue

            df = self.load(label)

            item = {}

            for result in df["results"]["bindings"]:
                for key in keys:
                    year = self.getYearIntValue(result, key)
                    if year:
                        item[key] = year

            results[label] = item

        os.makedirs(os.path.dirname(final_output_path), exist_ok=True)

        with open(final_output_path, "w") as f:
            json.dump(results, f, indent=4, ensure_ascii=False, sort_keys=True)


# labels_path = "people.json"
# tmp_dir = "../data"
# final_output_path = "../data/birth.json"

ins = WikidataTest(labels_path, tmp_dir, final_output_path)
ins.main()

100%|██████████| 5/5 [00:01<00:00,  3.97it/s]
